# install & import dependencies 

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
tf.__version__

'2.2.0-rc3'

# load data

In [0]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteANN9RA/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteANN9RA/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteANN9RA/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


# split the dataset into training & testing subsets

In [0]:
train_dataset, test_dataset = dataset['train'], dataset['test']

# preprocess

In [0]:
# define tokenizer 
tokenizer = info.features['text'].encoder

In [0]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

# shuffle the data to reduce bias
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
# pad the datasets into a uniform input_shape for the model
train_dataset = train_dataset.padded_batch(BATCH_SIZE, train_dataset.output_shapes)
test_dataset = test_dataset.padded_batch(BATCH_SIZE, test_dataset.output_shapes)

Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.


Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.


# define the model

In [0]:
model = keras.Sequential([
  # embeddings: after words are clustered, words found together are given similar vectors
    # vectors: magnitude and direction of multiple points
  # after model training, words in future sentences could have their direction as positive or negative
  # 64 = embedding_dimension                          
  keras.layers.Embedding(tokenizer.vocab_size, 64),
  # bidirectional= flowing 'cell state' goes both directions
  # return_sequences= when stacking LSTM layers, feeding an LSTM into another
    # requires the output of LSTM(current) to match the desired input of LSTM(next)
  keras.layers.Bidirectional(keras.layers.LSTM(units=64, return_sequences=True)),
  keras.layers.Bidirectional(keras.layers.LSTM(units=32)),
  # use a fully-connected Dense layer to map inputs to outputs/optimize weights
  # relu: return if x > 0, else return 0
  keras.layers.Dense(units=64, activation='relu'),
  # binary classification, return prediction 0 or 1 depending on higher probability
  keras.layers.Dense(units=1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          523840    
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 635,329
Trainable params: 635,329
Non-trainable params: 0
_________________________________________________________________


# compile the model 

build the model by compiling it with a loss, optimizer, and objective metric

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# train

fit the model to train & learn the relationships 

assign train model to a history object for querring

In [0]:
num_epochs = 10

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') >= .95):
      print('accuracy is greater than 95%, stopping training')
      self.model.stop_trianing=True

callbacks = myCallback()

# .fit(training_datasets, epochs, validation_datasets)
history = model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)


Epoch 1/10
391/391 [==============================] - 135s 345ms/step - loss: 0.6006 - accuracy: 0.6675 - val_loss: 0.5931 - val_accuracy: 0.6835
Epoch 2/10
391/391 [==============================] - 138s 352ms/step - loss: 0.5566 - accuracy: 0.6934 - val_loss: 0.6640 - val_accuracy: 0.6707
Epoch 3/10
391/391 [==============================] - 140s 357ms/step - loss: 0.4974 - accuracy: 0.7636 - val_loss: 0.4792 - val_accuracy: 0.7893
Epoch 4/10
391/391 [==============================] - 139s 357ms/step - loss: 0.3009 - accuracy: 0.8785 - val_loss: 0.3407 - val_accuracy: 0.8576
Epoch 5/10
391/391 [==============================] - 139s 356ms/step - loss: 0.2168 - accuracy: 0.9182 - val_loss: 0.3277 - val_accuracy: 0.8716
Epoch 6/10
391/391 [==============================] - 139s 355ms/step - loss: 0.1696 - accuracy: 0.9398 - val_loss: 0.3722 - val_accuracy: 0.8626
Epoch 7/10
391/391 [==============================] - 139s 354ms/step - loss: 0.1308 - accuracy: 0.9563 - val_loss: 0.3872 -

# visualize history performance

In [0]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel('epochs')
  plt.ylabel(string)
  plt.legend(string, 'val_'+string)
  plt.show()

plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

# clean up

terminate the memory kernel and free up the memory resources

In [0]:
import os, signal

# os.kill(os.getpid(), signal.SIGKILL)